# Introduction

#### https://www.kaggle.com/code/lucamassaron/eda-target-analysis
を初学者が初学者なりに解釈していく。

間違いがあれば指摘して頂けると幸いです。

Given the large number of features and groups present in the data, gaining insight on how the data is structured will bring an advantage when modelling it (hint: don't rush to build massive models treating all the data in the same way).

データに含まれる特徴量やグループの数が多い場合、データ構造の把握することは、モデリングする際に有利になる。
（ヒント：すべてのデータを同じように扱う大規模なモデルの構築を急がない）。

*Version history and highlights*

vers. 01 : adding a first target analysis and making the notebook public

vers. 02 : added more comparative plots

vers. 03 : added a scatter of time and assets and a brief investigation on features

vers. 04 : more on relationship between target and time

*バージョンの履歴とハイライト*

バージョン 01 : 最初のターゲット分析の追加とノートブックの公開

バージョン 02 : より多くの比較プロットを追加

バージョン 03 : 時間(time)と資産(assets)の散布図と特徴に関する簡単な調査を追加

バージョン 04 : ターゲット(target)と時間(time)の関係についての詳細

In [ ]:
# モジュールのインポート

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# trainデータのインポート

# featherはpandasのDataFrameを高速に保存、ロードするために開発されたApache Arrowのpythonラッパーだそう
# Apache Arrowはカラム型のフォーマット

train = pd.read_feather('../input/training-data-to-feather-python-r-low-mem/train.feather')

# A first look over the data

Getting an idea of how many observations, assets and time steps

In [ ]:
# trainデータ数の把握

obs = train.shape[0]
print(f"number of observations: {obs}")

In [ ]:
# time_id, investment_idのユニーク数の把握
# assets = investment_idとしているみたい

time_steps, assets = train.time_id.nunique(), train.investment_id.nunique()
print(f"number of assets: {assets} \t time steps: {time_steps}")

In [ ]:
# investment_id(assets)数に対するinvestment_idの範囲を表示

print(f"number of assets: {assets} (range from {train.investment_id.min()} to {train.investment_id.max()})")

The range of assets is more extended than the number fo assets themselves. In fact, assets will change in part in the test set, therefore you have to consider strategies for handling the ones in the training set in a generalizable way.
See: https://www.kaggle.com/c/ubiquant-market-prediction/discussion/301693#1656092

assetsの数よりも、範囲の方が広がっている。実際、test setではassetsが一部変更されるため、training setを一般化できるように処理する戦略を考える必要があるらしい。

# Target analysis

In [ ]:
# waiwai
# trainの中身見たくて追加
train.head()

In [ ]:
# waiwai
# groupbyがよくわからなくなってきたので確認
# 同じ値を持つデータをまとめて、それぞれの塊に対して共通の操作を行いたい時に使う。
obs_by_asset_tmp = train.groupby(['investment_id']).count()
obs_by_asset_tmp

In [ ]:
# investment_id毎にtargetの数をカウントしグラフ表示
# x軸はtarget数
# target数(取引回数？)に対するinvetment_id(assets)数を表示している

obs_by_asset = train.groupby(['investment_id'])['target'].count()

fig, ax = plt.subplots(1, 1, figsize=(12, 6))
obs_by_asset.plot.hist(bins=60)
plt.title("target by asset distribution")
plt.show()

If you more carefully look at the fabric of assets by time, you will notice, quite a few discontinuitites and that the discontinuities are more present in the first section of the time.

もっと注意深く、時間ごとの資産(assets)構成を見てみると、かなりの数の不連続性があり、その不連続性は時間の最初のセクションに多く存在することに気づく。

(waiwai-上記は何を意味しているかよくわかってない)

(waiwai-target数毎のinvestment_id(assets)の構成を見ると、target数が多いほど、investment_id数も増えるってこと？)

In [ ]:
# waiwai
# obs_by_assetの中身見たくて追加
obs_by_asset

In [ ]:
# waiwai
# train[['investment_id', 'time_id']]

In [ ]:
# x軸time_id, y軸investment_idの散布図を表示
train[['investment_id', 'time_id']].plot.scatter('time_id', 'investment_id', figsize=(20, 30), s=0.5)
plt.show()

Assets are distributed in a different way, there are assets that are actually more frequently observed and others that are not. **A good cv and modelling strategy should keep this into account** (stratify if you are working with subsamples).

assets(investment_id)の分布の仕方は様々で、実際に観測される頻度が高いassetsとそうでないassetsがある。(白いところは観測されていないところか、またtime_idが小さいほど観測されない頻度が大きい。time_id400前後に全く観測されないときがある。）

**良いCV(交差検証)とモデリング戦略はこれを考慮に入れるべきです** (サブサンプルを使っている場合は層別化する)。

In [ ]:
# investment_id毎にtargetを平均化しヒストグラムで表示
# x軸-target平均、y軸-investment_idの数

mean_target = train.groupby(['investment_id'])['target'].mean()
mean_mean_target = np.mean(mean_target)

fig, ax = plt.subplots(1, 1, figsize=(12, 6))
mean_target.plot.hist(bins=60)
plt.title("mean target distribution")
plt.show()

print(f"Mean of mean target: {mean_mean_target: 0.5f}")

The average of mean target by asset show a bell-shaped distribution, beware that there are outliers, anyway, because there are some assets with quite negative average target (-0.4 area) and some quite positive ones (+0.8 area). Overall the average mean target by asset is slightly negative (-0.0231)

資産(assets)別の平均目標値(target)を見ると、ベル型の分布を示しているが、平均目標値がかなりマイナスの資産（-0.4エリア）とかなりプラスの資産（+0.8エリア）があるため、とにかく異常値があることに注意しよう。全体として、資産別の平均目標値はわずかにマイナス（-0.0231）である。

In [ ]:
# investment_id毎にtargetの標準偏差をヒストグラムで表示
# x軸-target標準偏差、y軸-investment_idの数

sts_target = train.groupby(['investment_id'])['target'].std()
mean_std_target = np.mean(sts_target)

fig, ax = plt.subplots(1, 1, figsize=(12, 6))
sts_target.plot.hist(bins=60)
plt.title("standard deviation of target distribution")
plt.show()

print(f"Mean of std target: {mean_std_target: 0.5f}")

Also the average of mean standard deviation (std) by asset presents some interesting patterns. First of all, it is skewed toward the right, with some assets having more std (up to 2.5). On the other side there are also some few assets with std almost at zero.

また、資産(asset)別の平均標準偏差（std）の平均は、興味深いパターンを示している。まず、右肩上がりになっており、一部の資産では標準偏差が大きくなっている（最大2.5）。一方、標準偏差がほぼゼロの資産もいくつか存在する。

In [ ]:
# 全investment_id毎のtarget数をx軸、target平均をy軸にグラフ表示
# sns.jointplotでヒストグラム付き散布図
# kind='reg': 散布図と回帰直線
# {joint, marginal}_kws: dicts
# Additional keyword arguments for the plot components.
#  joint_kws={'line_kws':{'color':'red'}}←回帰直線を赤色にしてるだけぽい

ax = sns.jointplot(x=obs_by_asset.values, y=mean_target, kind="reg", 
                   height=8, joint_kws={'line_kws':{'color':'red'}})
ax.ax_joint.set_xlabel('observations')
ax.ax_joint.set_ylabel('mean target')
plt.show()

By jointly plotting the distribution of observartions by asset and the mean target value by asset, we may notice that the target value slightly reduces proportionally to the number of observation. The dispersion of values tends to grow with less observations, hence we need to re-plot the scatterplot this time using the standard deviation.

資産(asset)別の観測量分布(target数)と資産別の目標値(target)平均を併記すると、目標値が観測回数に比例してわずかに減少していることに気づくかもしれません。

(waiwai-観測関数に比例してわずかに増加していない？？)

値の分散は観測回数が少ないほど大きくなる傾向があるので、今度は標準偏差を使って散布図を再プロットする必要があります

In [ ]:
# 全investment_id毎のtarget数をx軸、target_stdをy軸にグラフ表示

qx = sns.jointplot(x=obs_by_asset.values, y=sts_target, kind="reg", 
                   height=8, joint_kws={'line_kws':{'color':'red'}})
ax.ax_joint.set_xlabel('observations')
ax.ax_joint.set_ylabel('std target')
plt.show()

The new scatterplot reveals that the less the observations, imply a much more uncertainty in the mean target.

この散布図から、(targetの)観測回数が少ないほど、平均的な目標(target)の不確かさが大きいことがわかる。

**Strategy**: in training you need to control this effect by expliciting the number of observations because this is predictive of the uncertainty of the predictions. In the test phase, instead, when you are working with an asset that you don't know about, you need to impute an average number of observations, thus expecting an average dispersion of predictions for that asset.

**戦略**
トレーニングでは，(targetの)観測数をインプットすることによって，この効果(観測数でターゲットの不確かさが増すこと)を制御する必要がある。なぜなら，これは(targetの)不確実性を予測するから。
テストフェーズでは、あなたが知らない資産(asset)を扱っているとき、観測数の平均数をインプットする必要がある。つまりその資産の予測値の平均分散を期待する。

(waiwai-assetが分かるときはtarget観測数により標準偏差が異なることに注意し、assetが分からないときは平均観測数とおいて標準偏差を計算するってこと？)

In [ ]:
# time_id毎にinvetment_idのユニーク数を表示

fig, ax = plt.subplots(1, 1, figsize=(12, 6))
train.groupby('time_id')['investment_id'].nunique().plot()
plt.title("number of unique assets by time")
plt.show()

As we have reasoned how the investments with less observations seem more risky, we notice how the number of the assets present at each time step is quite different and also highly oscillating. By the end of the avaliable time, the number of assets has grown by one third.

(targetの)観測回数が少ない投資(asset)ほどリスクが高いことを説明したが、各時間ステップに存在する資産の数が全く異なり、しかも大きく振動していることに気がつく。利用可能時間(trainに含まれるtime_id)が終了するまでに、資産(asset)の数は3分の1増えている。

In [ ]:
# time_id毎にinvetment_idのユニーク数を表示
# time_id毎にtargetの平均を表示(赤線は全time_idのtarget平均)
# time_id毎にtargetのstdを表示(赤線は全time_idのstd平均)

plt.figure(figsize=(12, 6))

plt.subplot(3, 1, 1,)
(train.groupby('time_id')['investment_id'].nunique()).plot()
plt.title("number of unique assets by time")

plt.subplot(3, 1, 2)
train.groupby('time_id')['target'].mean().plot()
plt.title("average target by time")
plt.axhline(y=mean_mean_target, color='r', linestyle='--', label="mean")
plt.legend(loc='lower left')

plt.subplot(3, 1, 3)
train.groupby('time_id')['target'].std().plot()
plt.title("std of target by time")
plt.axhline(y=mean_std_target, color='r', linestyle='--', label="mean")
plt.legend(loc='lower left')

plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=1.3, 
                    wspace=0.4, 
                    hspace=0.4)

plt.show()

In [ ]:
# invetment_idのユニーク数とtargetの平均の相関を算出
# corrcoef

r = np.corrcoef(train.groupby('time_id')['investment_id'].nunique(), train.groupby('time_id')['target'].mean())[0][1]
print(f"Correlation of number of assets by target: {r:0.3f}")

In [ ]:
# corrcoef使用時に[0][1]をつけている理由の確認
# 相関係数が2行2列で出てくるため、invetment_idのユニーク数とtargetの平均の相関は[0][1]or[1][0]
r_tmp = np.corrcoef(train.groupby('time_id')['investment_id'].nunique(), train.groupby('time_id')['target'].mean())
print(r_tmp)

If we plot the number of assets by time alongside the average target by time, it becomes evident that when there are less assets, the target oscillates more with prevalently higher targets. The correlation of assets number and target is negative, in fact. I wonder if we are modelling the asset allocation strategies alongside the markets. 

時間ごとの資産(asset)数と時間ごとの平均目標(target)値を並べてみると、資産数が少ないほど目標値の変動が大きく、目標値が高くなる傾向があることがわかる。
(waiwai-time_id400前後のあたりのことを言っている？)

実際、資産数と目標値の相関は負である。アセットアロケーション戦略(運用する資金を国内外の株や債券などにどのような割合で投資するのか決めること)をマーケットと一緒にモデル化しているのだろうか。

Let's now observe more closely the relationship between target and time:

In [ ]:
# x軸time_id, y軸targetのグラフを表示
# 薄い青色がtarget平均±1σ

time2target_mean = train.groupby(['time_id'])['target'].mean()
time2target_std = train.groupby(['time_id'])['target'].std()

_, axes = plt.subplots(1, 1, figsize=(24, 12))

# plt.fill_between(x, y1, y2):でy1とy2の間を塗りつぶす
plt.fill_between(
        time2target_mean.index,
        time2target_mean - time2target_std,
        time2target_mean + time2target_std,
        alpha=0.1,
        color="b",
    )

plt.plot(
        time2target_mean.index, time2target_mean, "o-", color="b", label="Training score"
    )
# plt.axhline: 水平線を描画する
plt.axhline(y=mean_mean_target, color='r', linestyle='--', label="mean")
axes.set_ylabel("target")
axes.set_xlabel("time")
plt.show()

Clearly the target is detrended and forced to mean zero and unit standard deviation, though at times when the number of assets is reduced the average changes and consequently shifts also the confidence interval. Being able to figure out such times of mean shift in the test set could be quite advantageous.

明らかにtargetはデトレンドされ、平均値ゼロ、単位標準偏差に強制されているが、資産(asset)数が減少すると平均値が変化し、その結果信頼区間(おそらく±1σ区間)も変化している。このような平均値の変化のタイミングをテストセットで把握することができれば、非常に有利になる。

Basically, this chart is the key. The task of the competition is to find out the position of an asset in a day. Is the asset near the average or how much is far away from it (you are predicting volatility, basically). In fact the evaluation is based on the mean of the Pearson correlation coefficient for each time ID.

基本的にはこのチャートがポイントになります。コンペの課題は、ある資産(asset)の1日のポジションを知ることです。その資産は平均に近いのか、それともどれくらい離れているのか（基本的にはボラティリティを予測していることになる）。実際には、各タイムIDのピアソン相関係数の平均値に基づいて評価されます。

In the following chart we are overimposing the target for asset 70 with the market average and the unit standard deviation band.

以下のチャートでは、資産(asset)70の目標値を市場平均(target平均のこと?)と単位標準偏差帯でオーバーインポーズ(重畳?)しています。

Clearly the position of asset 70 depends on its performance but also on the way the mean and standard deviation for that period_id is calculated (are we analyzing the volatility inside a basket of investment, maybe?).

明らかに資産70の位置はそのパフォーマンスに依存しますが、そのperiod_idの平均と標準偏差の計算方法にも依存します（投資のバスケット内部のボラティリティを分析しているのでしょうか、多分？）

(waiwai- 最後の文章はよくわからない) 

In [ ]:
# 上記グラフにinvestment_id70のtime_id毎のtargetを重畳させる

time2target_mean = train.groupby(['time_id'])['target'].mean()
time2target_std = train.groupby(['time_id'])['target'].std()

_, axes = plt.subplots(1, 1, figsize=(24, 12))
plt.fill_between(
        time2target_mean.index,
        time2target_mean - time2target_std,
        time2target_mean + time2target_std,
        alpha=0.1,
        color="b",
    )
plt.plot(
        time2target_mean.index, time2target_mean, "o-", color="b", label="Training score"
    )
plt.axhline(y=mean_mean_target, color='r', linestyle='--', label="mean")

asset = 70
plt.plot(train[train.investment_id==asset].time_id,
               train[train.investment_id==asset].target, '.')

axes.set_ylabel("target")
axes.set_xlabel("time")
plt.show()

**Strategy**: now your cv strategy should be clear, you have to do groupkfold on the time_id, keeping all the assets realtive to a time_id or in train or in validation.

**戦略**：CVの戦略は明確で、すべてのassetをtime_idに実在させるか、trainまたはvalidationで維持しtime_idでgroupkfoldを行う必要があります。

(waiwai- 全てのtime_idに全てのassetが存在するわけではないから、なんらかの手法で穴埋め後にGroupkfold(CV)する必要があるってこと？)

# Hypothesis: we can get a proxy of the count of obs by asset based on the features?

仮説：特徴量から資産(asset)別のtime_id観測数の変わりを得ることができる？

In [ ]:
# target変数にinvestment_id毎のtarget観測数を格納

# investment_id-target観測数の辞書型変数
obs_by_asset = train.groupby(['investment_id'])['target'].count().to_dict()
# print(obs_by_asset)

# trainデータに含まれるinvestment_idを抜き出した後、target観測数に置換
target = train.investment_id.copy().replace(obs_by_asset).astype(np.int16)
# print(target)

features = train.columns[4:]
del(obs_by_asset)

In [ ]:
# target観測数と特徴量の相関を算出

corrs = list()
for col in features:
    corr = np.corrcoef(target, train[col])[0][1]
    corrs.append(corr)
    
del(target)

In [ ]:
# 相関の高いもの20個を表示

# Series構造に相関値を変換
# ≒NumPyのndarrayの一次元配列(indexを番号以外でつけることができる)
feat_importances = pd.Series(corrs, index=features)
feat_importances.nlargest(20).plot(kind='barh', figsize=(12, 6)).invert_yaxis()
plt.show()

Actually based on a few features, the fact that an asset has less or more observations should be quite predictable based on features with high correlation. 

実際にいくつかの特徴量に基づいて、ある資産(asset)の観測値が少ないか多いかという事実は、高い相関を持つ特徴量に基づいてかなり予測できるはずです。
(f_164, f_22, f_61あたりは結構相関高め)

## If liked the notebook and you found it useful, please consider to upvote :-)

もしノートブックが有用だと思ったら、upvoteを検討してください:-) 